# Bundesrat Repraesentation

- Bundesratszusammensetzung pro Jahr
- Demografische Kennzahlen müssen entsprechend extrapoliert werden


In [1]:
# import libraries
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
from datetime import datetime
# import re
import os
# import json
# import requests

In [2]:
# import list of federal council from wikipedia de and wikipedia en
link = "https://de.wikipedia.org/wiki/Liste_der_Mitglieder_des_Schweizerischen_Bundesrates"
bundesraete = pd.read_html(link,header=0)[0]
link = "https://en.wikipedia.org/wiki/List_of_members_of_the_Swiss_Federal_Council"
federal_council = pd.read_html(link, header=0)[0]

In [3]:
council_merge = pd.merge(bundesraete, federal_council[["Name", "Election", "Left office", "Party", "Seq[6]"]], on="Name", how="left")

In [4]:
council_merge[pd.isna(council_merge.Election)]

,Jahr,Bild,Name,Lebensdaten,Kanton,Partei,Election,Left office,Party,Seq[6]
9,1855–1875,NaN,Josef Martin Knüsel,1813–1889,Luzern,freis.,NaN,NaN,NaN,NaN
35,1900–1912,NaN,Marc Ruchet,1853–1912,Waadt,FDP,NaN,NaN,NaN,NaN


In [5]:
council_merge.loc[council_merge.Name == "Josef Martin Knüsel", "Election"] = "14 July 1855"
council_merge.loc[council_merge.Name == "Josef Martin Knüsel", "Left office"] = "31 December 1875"
council_merge.loc[council_merge.Name == "Josef Martin Knüsel", "Party"] = "FDP/PRD"
council_merge.loc[council_merge.Name == "Josef Martin Knüsel", "Seq[6]"] = "D02"

council_merge.loc[council_merge.Name == "Marc Ruchet", "Election"] = "14 December 1899"
council_merge.loc[council_merge.Name == "Marc Ruchet", "Left office"] = "9 July 1912"
council_merge.loc[council_merge.Name == "Marc Ruchet", "Party"] = "FDP/PRD"
council_merge.loc[council_merge.Name == "Marc Ruchet", "Seq[6]"] = "G05"

In [94]:
council_merge.Election = council_merge.Election.apply(lambda x: datetime.strptime(x, "%d %B %Y"))
council_merge["Left office"] = council_merge["Left office"].apply(lambda x: "amtierend" if (x == "Incumbent") else datetime.strptime(x.split("[")[0], "%d %B %Y"))

In [95]:
council_merge.drop(["Bild", "Partei"], axis=1, inplace=True)

In [4]:
# Export data to manually edit
council_merge.to_csv('internal/data_edit.csv', index=False)

In [5]:
# Import edited data
council_merge = pd.read_csv('internal/data_edit.csv')

In [9]:
council_merge.iloc[0,0] = "kjsdhfkjasdhfkjash"

---

## Create year-by-year data

In [10]:
data = pd.DataFrame()
i = 0
for year in range(1848,2019):
    data.loc[i,"year"] = year
    i += 1

### Liste der Bundesräte um Metadaten ergänzen:
    - Alter in entsprechendem Jahr
    - Ausbildung
    - Wohnort (Ort & Raumgliederung)
    - Geschlecht
    - Zivilstand
    - Beruf
    - Religion
    - 

In [40]:
bundesraete

,Jahr,Name,Lebensdaten,Kanton,Partei,Eintritt,Austritt
0,1848–1861,Jonas Furrer,1805–1861,Zürich,freis.,1848,1861
1,1848–1854,Ulrich Ochsenbein,1811–1890,Bern,freis.,1848,1854
2,1848–1855,Henri Druey,1799–1855,Waadt,freis.,1848,1855
3,1848–1855,Josef Munzinger,1791–1855,Solothurn,freis.,1848,1855
4,1848–1857,Stefano Franscini,1796–1857,Tessin,freis.,1848,1857
5,1848–1866,Friedrich Frey-Herosé,1801–1873,Aargau,freis.,1848,1866
6,1848–1875,Wilhelm Matthias Naeff,1802–1881,St. Gallen,freis.,1848,1875
7,1855–1863,Jakob Stämpfli,1820–1879,Bern,freis.,1855,1863
8,1855–1867,Constant Fornerod,1819–1899,Waadt,freis.,1855,1867
9,1855–1875,Josef Martin Knüsel,1813–1889,Luzern,freis.,1855,1875


### Demographische Kennzahlen der CH-Bevölkerung sammeln
Besprechung mit Barney:
- Alter
- Wähleranteile
- Geschlecht
- Herkunft (Reich / Arm)
- Kanton
- Beruf (diveristät)
- (Städtisch / perubian / ländlich)
    
Sinnvolle Zeiträume?
- 1848? 
- 1919: ersten Proporzwahlen?
- 1971: Einführung des Frauenstimmrechts?



### Altersklassen
- https://hsso.ch/de/2012/b/8a

In [50]:
alter = pd.read_excel("input/B.08a.xlsx", skiprows=3)
alter = alter[3:17]
# extrapolieren

### Parteistärken
- https://hsso.ch/de/2012/x/6

In [7]:
parteistaerke = pd.read_excel("input/X.06.xlsx", skiprows=16, header=None)

In [11]:
parteistaerke.columns = ["Jahr", "Radikale/FDP", "Lib.-dem.", "CVP", "Konservative", "EVP", "soz.-pol., Demokr",
                         "Sozialdemokraten", "BGB/SVP", "Kommunisten / PdA.", "LdU", "div.", "Nation. Aktion / SD",
                         "POCH / GPS", "Autopartei", "Lega", "Total", "(leer)", "SR_Radikale/FDP", "SR_Lib.-dem.", "SR_CVP",
                         "SR_Konservative", "SR_soz.-pol., Demokr", "SR_Sozialdemokraten", "SR_BGB/SVP", "SR_LdU",
                         "SR_div.", "SR_Total"]

In [12]:
parteistaerke

,Jahr,Radikale/FDP,Lib.-dem.,CVP,Konservative,EVP,"soz.-pol., Demokr",Sozialdemokraten,BGB/SVP,Kommunisten / PdA.,...,SR_Radikale/FDP,SR_Lib.-dem.,SR_CVP,SR_Konservative,"SR_soz.-pol., Demokr",SR_Sozialdemokraten,SR_BGB/SVP,SR_LdU,SR_div.,SR_Total
0,1848,79,11,10,5.0,NaN,6.0,NaN,NaN,NaN,...,30,8,6,0.0,NaN,NaN,NaN,NaN,NaN,44
1,1851,78,16,16,7.0,NaN,3.0,NaN,NaN,NaN,...,27,7,8,2.0,NaN,NaN,NaN,NaN,NaN,44
2,1854,80,16,14,6.0,NaN,2.0,NaN,NaN,NaN,...,28,6,8,1.0,NaN,NaN,NaN,NaN,1.0,44
3,1857,80,15,20,5.0,NaN,0.0,NaN,NaN,NaN,...,25,9,9,0.0,NaN,NaN,NaN,NaN,1.0,44
4,1860,64,37,15,3.0,NaN,1.0,NaN,NaN,NaN,...,18,13,11,2.0,NaN,NaN,NaN,NaN,0.0,44
5,1863,59,37,21,5.0,NaN,6.0,NaN,NaN,NaN,...,14,16,13,1.0,NaN,NaN,NaN,NaN,0.0,44
6,1866,53,39,21,4.0,NaN,11.0,NaN,NaN,NaN,...,15,18,10,0.0,NaN,NaN,NaN,NaN,1.0,44
7,1869,56,31,23,3.0,NaN,15.0,NaN,NaN,NaN,...,16,16,11,1.0,NaN,NaN,NaN,NaN,0.0,44
8,1872,60,27,30,3.0,NaN,15.0,NaN,NaN,NaN,...,19,9,16,0.0,NaN,NaN,NaN,NaN,0.0,44
9,1875,63,22,33,2.0,NaN,15.0,NaN,NaN,NaN,...,19,9,16,0.0,NaN,NaN,NaN,NaN,0.0,44


### Herkunft (reich / arm)

Historische Daten schwer zu finden:
Alternativ können für die ersten 50 Jahre die Daten des World-Gini-Index genutzt werden. (https://en.wikipedia.org/wiki/Gini_coefficient#World_income_Gini_index_since_1800s)

- 